In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd

# Charger le fichier XML
tree = ET.parse('mon_fichier.xml')
root = tree.getroot()

# Charger le DataFrame
df = pd.read_csv('mon_dataframe.csv')

# Pour chaque article dans le XML
for article in root.findall('article'):

    # Récupérer l'article_id
    article_id = article.get('old-id')

    # Vérifier si l'article_id est présent dans le DataFrame
    if article_id in df['article_id'].values:

        # Récupérer la ligne correspondante dans le DataFrame
        df_row = df.loc[df['article_id'] == article_id]

        # Vérifier si localisationco est "none" et insee_code est présent
        if df_row['localisationco'].values[0] == 'none' and pd.notnull(df_row['insee_code'].values[0]):

            # Ajouter l'attribut type="commune" à la balise article
            article.set('type', 'commune')

            # Ajouter la balise <insee> à la balise article avec le code INSEE
            insee_code = df_row['insee_code'].values[0]
            insee_elem = ET.Element('insee')
            insee_elem.text = insee_code
            article.append(insee_elem)

        # Vérifier si localisationco est présent et insee_code est présent
        elif pd.notnull(df_row['localisationco'].values[0]) and pd.notnull(df_row['insee_code'].values[0]):

            # Ajouter le code INSEE à la balise <commune>
            insee_code = df_row['insee_code'].values[0]
            commune_elem = ET.Element('commune', insee=insee_code)
            commune_elem.text = df_row['localisationco'].values[0]
            
            # Ajouter la balise <localisationco> à la balise article avec le code INSEE et le nom de la commune
            loc_elem = ET.Element('localisationco')
            loc_elem.append(commune_elem)
            article.append(loc_elem)

# Enregistrer le fichier XML modifié
tree.write('mon_fichier_modifie.xml', encoding='UTF-8', xml_declaration=True)